<a href="https://colab.research.google.com/github/JHyunjun/SNU/blob/main/Graph%20Convolutional%20Network2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Convolutional Network - Lab 2

## Data Preprocessing

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/snu/w4/실습자료/7-29 gcn-cora/cora/cora.content'
cora_content = pd.read_csv(path, sep='\t', header=None)
cora_content.head()

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [4]:
ids = cora_content[0].values # paper(node) ids
vecs = cora_content.iloc[:, 1:1434].values # node features
labels = cora_content[1434].values # node labels

for l in np.unique(labels):
    print(l, labels[labels == l].shape[0])

Case_Based 298
Genetic_Algorithms 418
Neural_Networks 818
Probabilistic_Methods 426
Reinforcement_Learning 217
Rule_Learning 180
Theory 351


In [5]:
# one hot encode node labels
labels_onehot = pd.get_dummies(labels)

print(labels_onehot[:5])

   Case_Based  Genetic_Algorithms  Neural_Networks  Probabilistic_Methods  \
0           0                   0                1                      0   
1           0                   0                0                      0   
2           0                   0                0                      0   
3           0                   0                0                      0   
4           0                   0                0                      1   

   Reinforcement_Learning  Rule_Learning  Theory  
0                       0              0       0  
1                       0              1       0  
2                       1              0       0  
3                       1              0       0  
4                       0              0       0  


In [6]:
inds = np.arange(ids.shape[0]) # use index at identifying each node
x = vecs
y = labels_onehot
print(ids.shape, x.shape, y.shape)

(2708,) (2708, 1433) (2708, 7)


In [7]:
from sklearn.model_selection import train_test_split

num_classes = 7
num_per_train = 10
num_per_test = 100

y_train, y_test, idx_train, idx_test = train_test_split(
    y, inds, stratify=y, random_state=42,
    train_size=num_classes * num_per_train,
    test_size=num_classes * num_per_test)

idx_train, idx_valid = train_test_split(
    idx_train, stratify=y_train, random_state=42,
    train_size=int(num_classes * num_per_train * 0.8),
    test_size=int(num_classes * num_per_train * 0.2))

print(idx_train.shape, idx_valid.shape, idx_test.shape)

(56,) (14,) (700,)


## Skeleton Codes

In [8]:
from tensorflow import sparse
# from tensorflow.layers import Dense
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model


In [13]:
class GCN2(Model):
    def __init__(self, indices, values, input_dim=1433, 
                 hid_dim=64, num_classes=7, num_nodes=2708,
                num_layers=2):
        super(GCN2, self).__init__()
        
        # Hyperparameters of a model      
        self.num_nodes = num_nodes
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.hid_dim = hid_dim    
        self.num_layers = num_layers
                
        self.indices = indices
        self.values = tf.cast(values, dtype='float32')
        
        # Define layers
        self.dense_layers = [Dense(self.hid_dim, kernel_initializer='he_normal', activation='relu') 
                             for _ in range(self.num_layers)]
        self.dense_layers.append(Dense(self.num_classes, kernel_initializer='he_normal'))
        
    def call(self, x):
        A_size = (self.num_nodes, self.num_nodes)
        A = sparse.SparseTensor(self.indices, self.values, A_size)

        L = tf.cast(x, 'float32')

        for l in range(self.num_layers) : 
          L_new = sparse.sparse_dense_matmul(A,L)
          L_new = self.dense_layers[l](L_new)
          if l > 0 : 
            L_new = L + L_new
          L = L_new
        return self.dense_layers[-1](L)
        
    def loss_fn(self,logits, labels, indices):
        _labels = tf.gather_nd(labels, indices)
        _logits = tf.gather_nd(logits, indices)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=_labels, logits=_logits)
        return tf.reduce_mean(loss)
    
    def evaluate(self, x, labels, indices):
        logits = self.call(x)
        loss = self.loss_fn(logits, labels, indices)        
        _logits = tf.gather_nd(logits, indices)
        _labels = tf.gather_nd(labels, indices)
 
        pred = tf.argmax(_logits, axis=1)
        ans = tf.argmax(_labels, axis=1)
        correct = tf.equal(pred, ans)
        acc = tf.reduce_mean(tf.cast(correct, tf.float32))
        return loss, acc
    
    def train(self, x, labels, idx_train, idx_val, optimizer, max_epochs=20):
        for epoch in range(1, max_epochs+1):
            with tf.GradientTape() as tape:
                logits = self.call(x)
                train_loss = self.loss_fn(logits, labels, idx_train)
            
            grad_list = tape.gradient(train_loss, self.weights)
            grads_and_vars = zip(grad_list, self.weights)
            optimizer.apply_gradients(grads_and_vars)
            
            # Evaluation
            train_loss, train_acc = self.evaluate(x, labels, idx_train)
            valid_loss, valid_acc = self.evaluate(x, labels, idx_val)
            print(f"Epoch {epoch:3d}: {train_loss:.4f}, {train_acc*100:.2f}," 
                  ,f"{valid_loss:.4f}, {valid_acc*100:.2f}")            

In [17]:
def get_adj_matrix(ids):
    num_nodes = ids.shape[0]
    cites = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/snu/w4/실습자료/7-29 gcn-cora/cora/cora.cites', dtype = np.int32)
    id_map = {v : u for u, v in enumerate(ids)}
    indices = [(e,e) for e in range(num_nodes)]

    for node1, node2 in cites : 
      if node1 != node2 : 
        idx1 = id_map[node1]
        idx2 = id_map[node2]
        indices.append((idx1, idx2))
        indices.append((idx2, idx1))
    indices = np.array(indices)
    values = np.ones(indices.shape[0])

    return indices, values

In [21]:
def normalize(indices, values, num_nodes, way='both'):
    values_sum = np.ones(num_nodes)
    for node1, node2 in indices : 
      values_sum[node1] += 1
      values_sum[node2] += 1

    if way == 'both' : 
      values /= np.sqrt(values_sum[indices[:,1]])
      values /= np.sqrt(values_sum[indices[:,0]])
    
    elif way == 'row' : 
      values /= values_sum[indices[:,0]]
    
    elif way == 'col' : 
      values /= values_sum[indices[:,1]]

    else : 
      raise ValueError()

    return values # normalized values 

In [22]:
num_nodes = ids.shape[0]
indices, values = get_adj_matrix(ids)
values = normalize(indices, values, num_nodes, way='both')

train_mask = np.expand_dims(idx_train, axis=1)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

gcn2 = GCN2(indices=indices, values=values,
           input_dim=x.shape[1], hid_dim=64, num_classes=num_classes, 
            num_nodes=x.shape[0], num_layers=2)
_idx_train = np.expand_dims(idx_train, axis=1)
_idx_val = np.expand_dims(idx_valid, axis=1)

gcn2.train(x=x, labels=y, idx_train=_idx_train, idx_val=_idx_val, optimizer=optimizer, max_epochs=20)

Epoch   1: 1.7670, 85.71, 1.8838, 42.86
Epoch   2: 1.5403, 85.71, 1.7915, 50.00
Epoch   3: 1.2744, 85.71, 1.6822, 50.00
Epoch   4: 1.0027, 87.50, 1.5737, 57.14
Epoch   5: 0.7506, 92.86, 1.4641, 64.29
Epoch   6: 0.5300, 98.21, 1.3492, 71.43
Epoch   7: 0.3504, 100.00, 1.2321, 71.43
Epoch   8: 0.2168, 100.00, 1.1241, 78.57
Epoch   9: 0.1257, 100.00, 1.0281, 71.43
Epoch  10: 0.0695, 100.00, 0.9431, 64.29
Epoch  11: 0.0375, 100.00, 0.8767, 64.29
Epoch  12: 0.0202, 100.00, 0.8337, 71.43
Epoch  13: 0.0112, 100.00, 0.8138, 71.43
Epoch  14: 0.0064, 100.00, 0.8127, 78.57
Epoch  15: 0.0037, 100.00, 0.8222, 78.57
Epoch  16: 0.0023, 100.00, 0.8375, 78.57
Epoch  17: 0.0014, 100.00, 0.8553, 78.57
Epoch  18: 0.0009, 100.00, 0.8740, 78.57
Epoch  19: 0.0006, 100.00, 0.8928, 78.57
Epoch  20: 0.0004, 100.00, 0.9113, 78.57


In [23]:
test_mask = np.expand_dims(idx_test, axis=1)
test_loss, test_acc = gcn2.evaluate(x=x, labels=y, indices=np.expand_dims(idx_test, axis=1))
print(test_acc)

tf.Tensor(0.67714286, shape=(), dtype=float32)
